### WORKSPACE STRUCTURE FOR PET DETECTION

pets_detection/
        ├─ data/
        │    ├── images/
        │    │      ├── pet (1).jpg
        │    │      ├── pet (2).jpg
        │    │      └── ...
        │    ├── annotations/
        │    │      ├── pet (1).xml
        │    │      ├── pet (2).xml
        │    │      └── ...
        |         |── train.record
        |    ├── test_images/
        │    │      ├── pet (10).jpg
        │    │      ├── pet (20).jpg
        │    │      └── ...
        │    ├── test_annotations/
        │    │      ├── pet (10).xml
        │    │      ├── pet (20).xml
        │    │      └── ...     
        |    |    |── test.record
        │    
        ├─ generate_tfrecord.py
        ├── pets_label_map.pbtxt
        ├── model_main_tf2.py
        ├── pipeline.config
        ├── ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
                ├── checkpoint
                ├── saved_model
                ├── pipeline.config     
        
        

### NECESSARY IMPORTS

In [1]:
import os
import tarfile
import shutil
import six.moves.urllib as urllib
from shutil import copyfile

### CREATING DIRECTORIES

In [2]:
# Directory for pets
directory = "pets_detection"
  
# Parent Directory path 
parent_dir = "/home/dkube/work/workspace"
  
# Path 
path = os.path.join(parent_dir, directory) 

os.mkdir(path) 
print("Directory '%s' created" %directory) 

Directory 'pets_detection' created


### Creating the workspace structure 

In [6]:
base_path = "/home/dkube/work/workspace/pets_detection"
data_path="data"

data_dir_path = os.path.join(base_path,data_path) 

os.mkdir(data_dir_path) 
print("Directory '%s' created" %data_path)

##### Creating the test-images and test-annotations folder ######

os.mkdir(data_dir_path+"/test_images") 
os.mkdir(data_dir_path+"/test_annotations")

Directory 'data' created


In [7]:
data_dir_path

'/home/dkube/work/workspace/pets_detection/data'

### Downloading the dataset 

In [8]:
def download(data_dir):
    opener = urllib.request.URLopener()
    opener.retrieve("http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz",data_dir +"/annotations.tar.gz")
    opener.retrieve("http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz", data_dir + "/images.tar.gz")
    print("Downloaded and saved the dataset. Location: ",data_dir)

In [9]:
download(data_dir_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Downloaded and saved the dataset. Location:  /home/dkube/work/workspace/pets_detection/data


### Extracting the downloaded files 

In [13]:
def extract(data_dir_path):
    print(data_dir_path)
    files = [os.path.join(data_dir_path, f) for f in os.listdir(data_dir_path) if f.endswith('tar.gz')]
    print(files)
    for filename in files:
        print(filename)
        tar = tarfile.open(filename)
        tar.extractall(data_dir_path)
        tar.close()
    print("Extracted objects and stored. Location: ",data_dir_path)

In [14]:
extract(data_dir_path)

/home/dkube/work/workspace/pets_detection/data
['/home/dkube/work/workspace/pets_detection/data/annotations.tar.gz', '/home/dkube/work/workspace/pets_detection/data/images.tar.gz']
/home/dkube/work/workspace/pets_detection/data/annotations.tar.gz
/home/dkube/work/workspace/pets_detection/data/images.tar.gz
Extracted objects and stored. Location:  /home/dkube/work/workspace/pets_detection/data


### Moving some images for testing 

In [15]:
def copy_images(source,destination):
    image_list=['Abyssinian_1.jpg','Abyssinian_10.jpg','Abyssinian_100.jpg']
    for i in image_list:
        copyfile(os.path.join(source,i),os.path.join(destination, i))

In [16]:
copy_images(data_dir_path+"/images",data_dir_path+"/test_images")

### Moving xml to test_annotations 

In [17]:
def copy_xml(source,destination):
    xml_list=['Abyssinian_1.xml','Abyssinian_10.xml','Abyssinian_100.xml']
    for i in xml_list:
        copyfile(os.path.join(source,i),os.path.join(destination, i))

In [18]:
copy_xml(data_dir_path+"/annotations/xmls",data_dir_path+"/test_annotations")

### Creating tensorflow records 

In [19]:
!python /home/dkube/work/workspace/pets_detection/generate_tfrecord.py -x /home/dkube/work/workspace/pets_detection/data/images -l /home/dkube/work/workspace/pets_detection/pet_label_map.pbtxt -o /home/dkube/work/workspace/pets_detection/data/annotations/train.record

Successfully created the TFRecord file: /home/dkube/work/workspace/pets_detection/data/annotations/train.record


In [20]:
!python /home/dkube/work/workspace/pets_detection/generate_tfrecord.py -x /home/dkube/work/workspace/pets_detection/data/test_images -l /home/dkube/work/workspace/pets_detection/pet_label_map.pbtxt -o /home/dkube/work/workspace/pets_detection/data/test_annotations/test.record

Successfully created the TFRecord file: /home/dkube/work/workspace/pets_detection/data/test_annotations/test.record


### Downloading the pretrained model 

In [21]:
opener = urllib.request.URLopener()
opener.retrieve("http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz",base_path +"/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz")

('/home/dkube/work/workspace/pets_detection/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
 <http.client.HTTPMessage at 0x7f3fb608f908>)

### Extracting the pretrained model 

In [22]:
tar = tarfile.open(base_path+'/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz')
tar.extractall(base_path)

### Changing the following things in pipeline config



#### Pipeline config file present in ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
 
1. num_classes: 37

2. fine_tune_checkpoint:"/home/dkube/work/workspace/pets_detection/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"

3. label_map_path:"/home/dkube/work/workspace/pets_detection/pet_label_map.pbtxt"

4. tf_record_input_reader { input_path:"/home/dkube/work/workspace/pets_detection/data/annotations/train.record"

5. eval_input_reader {label_map_path:"/home/dkube/work/workspace/pets_detection/pet_label_map.pbtxt"
shuffle: false
num_epochs: 1
  
 tf_record_input_reader     {input_path:"/home/dkube/work/workspace/pets_detection/data/test_annotations/test.record"
    
6. use_bfloat16: false

### Creating a new directory for our model checkpoints

In [6]:
os.mkdir(base_path+"/my_ssd_resnet50_v1_fpn")

In [7]:
shutil.copy('/home/dkube/work/workspace/pets_detection/pipeline.config','/home/dkube/work/workspace/pets_detection/my_ssd_resnet50_v1_fpn/')

'/home/dkube/work/workspace/pets_detection/my_ssd_resnet50_v1_fpn/pipeline.config'

### Training the model 

In [ ]:
!python /home/dkube/work/workspace/pets_detection/model_main_tf2.py --model_dir=/home/dkube/work/workspace/pets_detection/my_ssd_resnet50_v1_fpn --pipeline_config_path=/home/dkube/work/workspace/pets_detection/my_ssd_resnet50_v1_fpn/pipeline.config


INFO:tensorflow:Initializing local devices since in-graph multi-worker training with `MirroredStrategy` is not supported in eager mode. TF_CONFIG will be ignored when when initializing `MirroredStrategy`.
I1015 11:47:25.657989 140384311572288 mirrored_strategy.py:302] Initializing local devices since in-graph multi-worker training with `MirroredStrategy` is not supported in eager mode. TF_CONFIG will be ignored when when initializing `MirroredStrategy`.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1015 11:47:26.359178 140384311572288 mirrored_strategy.py:341] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1015 11:47:26.364014 140384311572288 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1015 11:47:26.364173 140384311572288 config_util.py:552] Maybe overwriting use_bfloat16: 